# Das DH Intro in Python

In diesem Tutorial-Notebook werden wir einige Teile des DH Intros mit Python umsetzen.

Erinnerung zur Notebook-Benutzung:
Ihr könnt Zellen mit dem Play-Button ausführen, oder mit Ctrl+Enter (Cmd+Enter für Mac?). Ihr könnt in Zellen hineinschreiben um sie zu modifizieren und Code auszuprobieren.

## Ordnermanagement und Datensatz herunterladen

In diesem ersten Abschnitt werden mithilfe von Python einen Ordner für unsere Dokumente anlegen, und dann über die API der Editionswebseite die XML-Dokumente herunterladen.

Wir können mit Python auf unserem Betriebssystem ähnlich wie im Terminal Ordner erstellen, löschen, Dateien kopieren, und alles weitere.

Für gewisse Funktionalitäten müssen wir in Python aber sogenannte *Packages* importieren. Wir unterscheiden dabei zwischen Packages der Standard-Bibliothek und externen Packages, die über *pip*, den Python Installer für Packages, installiert werden müssen. Packages der Standard-Bibliothek wurden zusammen mit Python bereits auf eurem Gerät installiert, damit wir ihre Funktionalität nutzen können, müssen sie aber jeweils noch *importiert* werden.

Um Ordner und Dateien zu managen gibt es in Python das [*os*-Package](https://docs.python.org/3/library/os.html).

In [ ]:
# Importieren des Package
import os

os.makedirs("docs", exist_ok=True)

# beachte das vorgestellte *os*, das anzeigt, dass wir die Funktion makedirs aus diesem Package verwenden.
# das erste Parameter sollte ein Dateipfad sein. makedirs kann auch gleich mehrere verschachtelte Ordner erstellen.
# falls exist_ok False wäre, würde eine Fehlermeldung ausgegeben werden, falls der Ordner bereits existiert.

Im nächsten Schritt laden wir uns denselben Datensatz herunter wie in Inas Tutorial, nämlich alle Briefe, die von Franz Marc geschrieben wurden.

Dazu gehen wir folgendermassen vor:
1. Den [Registereintrag](https://sturm-edition.de/register/personen/P.0000003.html) von Franz Marc beziehen, aber die API-Version.
2. Darin alle Briefe auslesen, die er geschrieben hat (mit FMA markiert).
3. Diese Briefe per API abfragen.

Um Abfragen an Webseiten zu schicken, eignet sich unkompliziert das [*requests*-Package](https://requests.readthedocs.io/en/latest/). Die Standard-Bibliothek enthält zwar ein Package zur Abfrage von URL, aber empfiehlt auch, besser *requests* zu verwenden.

In [ ]:
# Installation eines externen Package (in einem Notebook)
%pip install -U requests

# Mit dem % führen wir statt einem Python-Command ein Terminal-Command innerhalb des Notebooks aus. Ihr könntet das auch im Terminal tun stattdessen.
# Dieser Command lädt nun requests herunetr und installiert es auf eurem Gerät.
# -U updatet das Package, falls es bereits installiert ist
# Eventuell müsst ihr das Notebook nochmal neu starten (Restart bei den Notebook-Commands), damit das Package importiert werden kann.

In [ ]:
# Importieren des Package
import requests

# url von Franz Marcs Eintrag
url = "https://sturm-edition.de/api/persons/P.0000003"

# eine Abfrage schicken
response = requests.get(url)

# Überprüfen, dass die Abfrage erfolgreich war
print(response)

# 200 => Erfolg

In [ ]:
# Die erhaltene Datei ansehen
print(response.text)

Wir haben hier eine XML-Datei erhalten. Unter *linkGrp* finden wir *ptr*-Elemente, die auf die relevanten Briefdateien verweisen. Wir wollen aber nur jene, die von Franz Marc geschrieben wurden, also filtern wir den Dateinamen noch auf "FMA".

Um XML zu verarbeiten, empfehle ich das externe Package [lxml](https://lxml.de/).

In [ ]:
# Installiere lxml
%pip install -U lxml

In [ ]:
# Importiere das ElementTree-Subpackage von lxml und benenne es "et" für den Rest des Codes
from lxml import etree as et

# Lese den Text der Abfrage ein, um einen ElementTree zu bauen
root = et.fromstring(response.text)

In [ ]:
# Suche nach allen ptr-Elementen, die "FMA" als text enthalten.
fma_letters = root.xpath("./tei:linkGrp/tei:ptr[contains(@target,'FMA')]", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})

print(len(fma_letters), "letters found.")

Schliesslich fragen wir die Briefe ab und legen sie in unser vorher angelegtes Verzeichnis, damit wir nicht jedes Mal die API abfragen müssen, wenn wir mit den Briefen arbeiten möchten.

In [ ]:
# Iteriere alle ptr-Elemente
for letter in fma_letters:

    # Mit get erhalten wir den Wert des Dateinamen-Attributs aus dem Element
    filename = letter.get("target")

    # Abfrage schicken für den Brief
    response = requests.get("https://sturm-edition.de/api/files/" + filename)

    # Schreiben des XML in eine Datei im Ordner von vorher
    with open("docs/" + filename, mode="w", encoding="utf8") as out:
        out.write(response.text)


Perfekt, alle Briefe sollten nun im "docs"-Ordner liegen!

## Suche mit Regex
In diesem Abschnitt replizieren wir kurz die Suche per regex innerhalb von Python.
Um mit Regex zu suchen, gibt es in Python das [*re*-Package](https://docs.python.org/3/library/re.html) der Standard-Bibliothek, oder das externe *regex*-Package, das weitere Funktionalität bietet (die man aber eher selten benötigt). Für unsere Zwecke hier reicht *re*.

In [ ]:
# Importiere re
import re

# Um es einfach zu testen, können wir z.B. den Text aller Briefe auslesen und diesen zusammennehmen, dann durchsuchen
# das glob-Modul holt uns alle Dateien innerhalb eines bestimmten Ordners
import glob
all_text = ""
for letter in glob.glob("docs/*.xml"):
    all_text += open(letter, mode="r", encoding="utf8").read()

# Suche nach verschiedenen Schreibweisen von "Grüsse", mit einer Flag um Gross- und Kleinschreibung zu ignorieren
matches = re.findall("Gr[u|ü][ß|ss]\w*", all_text, flags=re.I)

# Welche Schreibweisen gibt es?
variants = set(matches)
print(variants)

Im Gegensatz zu Ina's Tutorial erhalten wir hier nicht die ganze Zeile. Aber wenn wir mehr Kontext möchten, z.B. die ganze Grussformel, dann tun wir das ohnehin besser indem wir die XML-Annotation nutzen.

## Suche im XML
Hier gehen wir nun einen Schritt weiter als im letzten Tutorial. Wir verwenden erneut lxml und suchen damit nach den Grussformeln im Schlussteil, die in dieser Edition einheitlich mit dem Tag *salute* im *closer* gekennzeichnet sind.

In [ ]:
# nochmal importieren, falls ihr das Notebook neu gestartet habt
from lxml import etree as et
import glob
import re

# wir sammeln alle Grussformeln in dieser Liste
salutations = []

for letter in glob.glob("docs/*.xml"):
    # diesmal lesen wir direkt eine Datei ein, keinen String
    tree = et.parse(letter)
    root = tree.getroot()
    
    # suche nach den salute-Events, aber nur im Textkörper
    salutes = root.xpath("./tei:text/tei:body/tei:div[@type='content']//tei:closer/tei:salute", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})

    for salute in salutes:
        # Aufräumen der Whitespaces
        salute_text = re.sub(r"\s*\n\s*", " ", salute.text)
        salutations.append(salute_text)

print(salutations)

In [ ]:
# Wie oft wird wie gegrüsst?
from collections import Counter

counts = Counter(salutations)

for salute, count in counts.most_common():
    print(salute, count)

### Noch ein letzter Hinweis

Ein wichtiger Hinweis noch zur Verwendung von *.text* um den Text aus XML-Elementen abzufragen. Hierbei ist zu beachten, dass dies nur den unmittelbaren Text erfasst, ohne verschachtelte Elemente zu beachten. Wenn z.B. der gesamte Text inklusive verschachtelten Elementen gewünscht ist, eignet sich *et.tostring(salute, method="text", encoding="utf8", with_tail=False).decode("utf8")* gut.